<a href="https://colab.research.google.com/github/uanushkatkd/CS6910_Assignment2/blob/main/cs6910_DL_2_partA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
print(torch.device('cuda:0'))
print(torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0
2.0.0+cu118
cpu


In [ ]:
!wget 'https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
!unzip -q nature_12K.zip

--2023-04-11 16:22:12--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.194.128, 173.194.195.128, 173.194.196.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  44.8MB/s    in 21s     

2023-04-11 16:22:33 (176 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [ ]:
# imports
import math
import torch
import torchvision
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import (
    DataLoader, random_split
)  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

from torchvision.datasets import ImageFolder
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import pathlib
torch.manual_seed(1) 
np.random.seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Dataset Augmentation 
def load_data(bs,augment_data=False):
    # define the transforms to be applied to the training data
    if augment_data:
        train_transforms = transforms.Compose([
          transforms.Resize((300,300)),
          transforms.RandomHorizontalFlip(),
          transforms.RandomRotation(10),
          transforms.ToTensor(),
          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ])
    else:
        train_transforms = transforms.Compose([
          transforms.Resize((300,300)),
          transforms.ToTensor(),
          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ])
    
    test_transforms = transforms.Compose([
      transforms.Resize((300,300)),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])


    home_path = "/content/inaturalist_12K"

    train_path = os.path.join(home_path,'train')
    test_path = os.path.join(home_path,'val')
    # define the dataset and apply the transforms
    train_dataset = ImageFolder(train_path, transform=train_transforms)
    test_dataset = ImageFolder(test_path, transform=test_transforms)

    # split training dataset into train and validation sets
    train_size = int(0.8 * len(train_dataset))
    print(train_size)
    val_size = len(train_dataset) - train_size
    print(val_size)

    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    # create a data loader for the training data
    train_loader = torch.utils.data.DataLoader(train_dataset,bs, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, bs, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, bs, shuffle=False)

    #categories
    root=pathlib.Path(train_path)

    classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

    return train_loader,val_loader,test_loader,classes





In [ ]:
#Simple CNN
def flatten(k=[11,9,7,5,3],w=300, s=1, p=1):
    r=w
    for i in  range(len(k)):
        print("r",r)
        r= (r+2*p-k[i])+1
        r= int(r/2)+1
    return r 


class CNN(nn.Module):
    def __init__(self, in_channels=3, num_class=10,num_filters=4,kernel_sizes=[11,9,7,5,3],fc_neurons=64,batch_norm=True,dropout=0.3,filter_multiplier=2,activation='LeakyRelu'):

        super(CNN, self).__init__()
        self.in_channels=in_channels
        self.num_class=num_class
        self.num_filters=num_filters
        self.kernel_sizes=kernel_sizes
        self.fc_neurons=fc_neurons
        self.activation=activation
        self.batch_norm=batch_norm
        self.dropout=dropout
        self.filter_multiplier=filter_multiplier
        
        #print("in1")
        self.conv1 = nn.Conv2d(3, num_filters, kernel_size=kernel_sizes[0],stride=1, padding=1).to(device)
        #print("in2")
        self.bn1=nn.BatchNorm2d(num_features=num_filters)
        self.relu1 = nn.LeakyReLU()

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.conv2 = nn.Conv2d(num_filters,filter_multiplier*num_filters,  kernel_size=kernel_sizes[1],stride=1, padding=1).to(device)
        self.bn2=nn.BatchNorm2d(num_features=filter_multiplier*num_filters)
        self.relu2 = nn.LeakyReLU()

        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.conv3 = nn.Conv2d(filter_multiplier*num_filters, int(math.pow(filter_multiplier, 2))*num_filters, kernel_size=kernel_sizes[2],stride=1, padding=1).to(device)
        self.bn3=nn.BatchNorm2d(num_features=int(math.pow(filter_multiplier, 2))*num_filters)
        self.relu3 = nn.LeakyReLU()


        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.conv4 = nn.Conv2d(int(math.pow(filter_multiplier, 2))*num_filters, int(math.pow(filter_multiplier, 3))*num_filters, kernel_size=kernel_sizes[3],stride=1, padding=1).to(device)
        self.bn4=nn.BatchNorm2d(num_features=int(math.pow(filter_multiplier, 3))*num_filters)
        self.relu4 = nn.LeakyReLU()

        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.conv5 = nn.Conv2d(int(math.pow(filter_multiplier, 3))*num_filters, int(math.pow(filter_multiplier, 4))*num_filters, kernel_size=kernel_sizes[4],stride=1, padding=1).to(device)
        self.bn5=nn.BatchNorm2d(num_features=int(math.pow(filter_multiplier, 4))*num_filters)
        self.relu5 = nn.LeakyReLU()


        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        print("ok pool5")
        self.r=flatten(kernel_sizes)
        print("ok flatten")
        print(self.r)
        self.fc1 = nn.Linear(in_features=int(math.pow(filter_multiplier, 4))*num_filters*self.r*self.r, out_features=fc_neurons)
        self.relu6 = nn.LeakyReLU()

        self.drop = nn.Dropout(dropout)
        
        self.fc2 = nn.Linear(in_features=fc_neurons, out_features=num_class)
      
    
    def forward(self, x):
        x = self.conv1(x)
        if self.batch_norm:
            x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.pool1(x)
        
        x = self.conv2(x)
        if self.batch_norm:
            x = self.bn2(x)
        x = self.relu2(x)

        x = self.pool2(x)

        x = self.conv3(x)
        if self.batch_norm:
            x = self.bn3(x)
        x = self.relu3(x)
        
        x = self.pool3(x)
        
        x = self.conv4(x)
        if self.batch_norm:
            x = self.bn4(x)
        x = self.relu4(x)
       
        x = self.pool4(x)
     
        x = self.conv5(x)
        if self.batch_norm:
            x = self.bn5(x)
        x = self.relu5(x)

        x = self.pool5(x)
        x = x.view(x.size(0),-1)

        x = self.fc1(x)
        x = self.relu6(x)
        x = self.drop(x)
        
        x = self.fc2(x)
        
        return x





In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channels = 3
num_class = 10
learning_rate = 0.0005
batch_size = 64
epochs = 15
data_aug=True

# Load data
train_loader,val_loader,test_loader,classes=load_data(batch_size,data_aug)
print(classes)
trainfeature, trainlabel = next(iter(train_loader))
print(f"Feature Batch Shape: {trainfeature.size()}")
print(f"Label Batch Shape: {trainlabel.size()}")



# Initialize network
model = CNN(3,10,16,[3,3,3,3,3],128,False,0,2,'LeakyRelu').to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer=optim.NAdam(model.parameters(),lr=learning_rate,weight_decay=0.0001)

# Train Network
for epoch in range(epochs):
    # Set the model to training mode
    model.train()

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        optimizer.zero_grad()
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        
    # Set the model to evaluation mode
    model.eval()

    # Track the total loss and number of correct predictions
    test_loss = 0
    num_correct = 0
    num_samples = 0

    # Evaluate the model on the validation set
    with torch.no_grad():
        for data, targets in test_loader:
            data = data.to(device=device)
            targets = targets.to(device=device)

            scores = model(data)
            test_loss += criterion(scores, targets).item()

            _, predictions = scores.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)

    # Calculate the average validation loss and accuracy
    test_loss /= len(test_loader)
    test_acc = float(num_correct) / num_samples

    # Print the epoch number, loss, and accuracy
    print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}, Test Acc: {:.2f}%'
          .format(epoch+1, epochs, loss.item(), test_loss, test_acc*100))

# Check accuracy on training & test to see how good our model
# Save best model
best_model_path = 'best_model.pth'
torch.save(model.state_dict(), best_model_path)
print(f"Best model saved to {best_model_path}")

7999
2000
['.DS_Store', 'Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Feature Batch Shape: torch.Size([64, 3, 300, 300])
Label Batch Shape: torch.Size([64])
ok pool5
r 300
r 151
r 76
r 39
r 20
ok flatten
11
Epoch [1/15], Train Loss: 2.1334, Test Loss: 2.1505, Test Acc: 21.35%
Epoch [2/15], Train Loss: 1.9480, Test Loss: 1.9612, Test Acc: 31.30%
Epoch [3/15], Train Loss: 2.0581, Test Loss: 1.9284, Test Acc: 32.50%
Epoch [4/15], Train Loss: 1.8285, Test Loss: 1.8730, Test Acc: 34.55%
Epoch [5/15], Train Loss: 1.8385, Test Loss: 1.8725, Test Acc: 34.05%
Epoch [6/15], Train Loss: 1.6753, Test Loss: 1.8064, Test Acc: 37.25%
Epoch [7/15], Train Loss: 1.7379, Test Loss: 1.7734, Test Acc: 36.40%
Epoch [8/15], Train Loss: 1.7229, Test Loss: 1.7558, Test Acc: 38.50%
Epoch [9/15], Train Loss: 1.5215, Test Loss: 1.7803, Test Acc: 37.50%
Epoch [10/15], Train Loss: 1.4917, Test Loss: 1.7439, Test Acc: 39.40%
Epoch [11/15], Train Loss:

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channels = 3
num_class = 10
learning_rate = 0.0001
batch_size = 32
epochs = 15
data_aug=False

# Load data
train_loader,val_loader,test_loader,classes=load_data(batch_size,data_aug)
print(classes)
trainfeature, trainlabel = next(iter(train_loader))
print(f"Feature Batch Shape: {trainfeature.size()}")
print(f"Label Batch Shape: {trainlabel.size()}")



# Initialize network
model = CNN(3,10,16,[7,5,5,3,3],64,True,0.2,2,'Mish').to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer=optim.NAdam(model.parameters(),lr=learning_rate,weight_decay=0.0001)

# Train Network
for epoch in range(epochs):
    # Set the model to training mode
    model.train()

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        optimizer.zero_grad()
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        
    # Set the model to evaluation mode
    model.eval()

    # Track the total loss and number of correct predictions
    test_loss = 0
    num_correct = 0
    num_samples = 0

    # Evaluate the model on the validation set
    with torch.no_grad():
        for data, targets in test_loader:
            data = data.to(device=device)
            targets = targets.to(device=device)

            scores = model(data)
            test_loss += criterion(scores, targets).item()

            _, predictions = scores.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)

    # Calculate the average validation loss and accuracy
    test_loss /= len(test_loader)
    test_acc = float(num_correct) / num_samples

    # Print the epoch number, loss, and accuracy
    print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}, Test Acc: {:.2f}%'
          .format(epoch+1, epochs, loss.item(), test_loss, test_acc*100))

# Check accuracy on training & test to see how good our model
# Save best model
best_model_path = 'best_model.pth'
torch.save(model.state_dict(), best_model_path)
print(f"Best model saved to {best_model_path}")

7999
2000
['.DS_Store', 'Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Feature Batch Shape: torch.Size([32, 3, 300, 300])
Label Batch Shape: torch.Size([32])
ok pool5
r 300
r 149
r 74
r 37
r 19
ok flatten
10
Epoch [1/15], Train Loss: 2.0479, Test Loss: 2.0485, Test Acc: 28.80%
Epoch [2/15], Train Loss: 1.9781, Test Loss: 1.9840, Test Acc: 30.55%
Epoch [3/15], Train Loss: 1.8830, Test Loss: 1.8953, Test Acc: 33.50%
Epoch [4/15], Train Loss: 1.5873, Test Loss: 1.8801, Test Acc: 34.70%
Epoch [5/15], Train Loss: 1.6135, Test Loss: 1.8147, Test Acc: 37.05%
Epoch [6/15], Train Loss: 1.8353, Test Loss: 1.8832, Test Acc: 34.05%
Epoch [7/15], Train Loss: 1.6605, Test Loss: 1.8454, Test Acc: 36.95%
Epoch [8/15], Train Loss: 1.2188, Test Loss: 1.8422, Test Acc: 37.30%
Epoch [9/15], Train Loss: 1.6026, Test Loss: 1.8820, Test Acc: 35.75%
Epoch [10/15], Train Loss: 1.8103, Test Loss: 1.8908, Test Acc: 37.45%
Epoch [11/15], Train Loss:

In [ ]:
#loading the best model and testing it on Test Data
best_model_path = 'best_model.pth'

loaded_model = CNN(3,10,16,[7,5,5,3,3],64,True,0.2,2,'Mish').to(device)
loaded_model.load_state_dict(torch.load(best_model_path)) # it takes the loaded dictionary, not the path file itself

def calculate_accuracy(model, test_loader,criterion):
    model.eval()
    total = 0
    correct = 0
    cost=0
    acc=0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            cost +=criterion(outputs,labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images
            del labels
    acc=100 * correct / total
    cost/=len(test_loader)
        
    return cost,acc
loss,acc=calculate_accuracy(loaded_model,test_loader,nn.CrossEntropyLoss())
print(loss,acc)

print(loaded_model.state_dict())


In [ ]:
best_model_path = 'best_model.pth'

loaded_model = CNN(3,10,16,[7,5,5,3,3],64,True,0.2,2,'Mish')
loaded_model.load_state_dict(torch.load(best_model_path)) # it takes the loaded dictionary, not the path file itself

# Initialize wandb
wandb.init(project="CS6910_Assignment_2_Q2")


# Define a function to generate predictions and sample images from the test data
def generate_predictions(model, data_loader):
    # Set the model to evaluation mode
    model.eval()
    
    # Create a list to store the predictions and sample images
    predictions = []
    sample_images = []
    
    # Generate predictions and sample images
    with torch.no_grad():
        for batch, _ in data_loader:
            # Forward pass through the model
            output = model(batch)
            
            # Get the predicted class labels
            _, predicted = torch.max(output, 1)
            
            # Convert the predicted labels to image tensors
            predicted_images = torchvision.utils.make_grid(batch[predicted])
            
            # Append the predictions and sample images to the lists
            predictions.append(predicted_images)
            sample_images.append(torchvision.utils.make_grid(batch))
    
    # Concatenate the predictions and sample images into grids
    prediction_grid = torchvision.utils.make_grid(predictions, nrow=3)
    sample_grid = torchvision.utils.make_grid(sample_images, nrow=3)
    
    # Return the grids
    return prediction_grid, sample_grid

# Generate the prediction and sample image grids
prediction_grid, sample_grid = generate_predictions(loaded_model, test_loader)

# Log the grids to wandb
wandb.log({
    'Predictions': wandb.Image(prediction_grid),
    'Sample Images': wandb.Image(sample_grid)
})

# Finish the run
wandb.finish()


In [ ]:

from signal import signal,SIGPIPE, SIG_DFL
signal(SIGPIPE,SIG_DFL)
!pip install wandb -qU
import wandb
!wandb login --relogin da816d14625ef44d200ee4acaa517646962e6f9a

wandb: Appending key for api.wandb.ai to your netrc file: /home/vplab/.netrc


In [ ]:

sweep_config = {
    "name" : "CS6910_Assignment_2_Q2",
    "method" : "bayes",
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    "parameters" : {
        "optimizer" : {
            "values" : ['adam','nadam','sgd']
        },
        "activation" : {
            "values" : ['LeakyRelu','Selu','Gelu','Mish']
        },
        "batch_size": {
            "values": [32, 64, 128]
        },
        'learning_rate':{
            "values": [0.001,0.0001,0.0003,0.0005]
        },
        "dropout": {
            "values": [0,0.2,0.3]
        },
        "batch_norm": {
              "values": [True,False]
        },
        "data_aug": {
              "values": [True,False]
        },
        'kernel_sizes':{
            'values': [[3,3,3,3,3],[5,5,5,5,5],[7,5,5,3,3], [11,9,7,5,3]]
        },
        'filter_multiplier': {
            'values': [1, 2, 0.5]
        },
        'num_filters': {
            'values': [4,8,16]
        },
        "fc_neurons": {
              "values": [32, 64, 128]
          }        
    }
}
def opti(model,opt='adam',lr=0.0005):
    print("in opti")
    if opt == "sgd":
        opt= optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif opt == "adam":
        opt = optim.Adam(model.parameters(),lr=lr,weight_decay=0.0001)
    elif opt == "nadam":
        opt = optim.NAdam(model.parameters(),lr=lr,weight_decay=0.0001)
    print('exit opti')    
    return opt

def calculate_accuracy(model, test_loader,criterion):
    model.eval()
    total = 0
    correct = 0
    cost=0
    acc=0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            cost +=criterion(outputs,labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images
            del labels
    acc=100 * correct / total
    cost/=len(test_loader)
        
    return cost,acc
def train():
    config_default={
      'epochs':15,
      'batch_size':32,
      'learning_rate':0.001,
      'dropout':0.3,
      'batch_norm':True,
      'data_aug':True,
      'kernel_sizes':[5,5,5,5,5],
      'filter_multiplier': 2,
      'num_filters': 16,
      "fc_neurons": 64
  }
    wandb.init(config=config_default)
    c= wandb.config
    name = "nfliter_"+str(c.num_filters)+"op_"+str(c.optimizer)+"_ac_"+str(c.activation)+"_n_"+str(c.learning_rate)+"_bs_"+str(c.batch_size)+"_dp_"+str(c.dropout)+"_bn_"+str(c.batch_norm)

    wandb.init(name=name)

    # Retrieve the hyperparameters from the config
    lr = c.learning_rate
    bs = c.batch_size
    epochs = 15
    act= c.activation
    opt= c.optimizer

    dp = c.dropout
    bn = c.batch_norm
    da=c.data_aug
    ks=c.kernel_sizes
    fm=c.filter_multiplier
    nf=c.num_filters
    fc=c.fc_neurons


    # Load the dataset
    train_loader,val_loader,test_loader,classes=load_data(bs,da)
    
    print("data loaded ====================================================")

    # Initialize network
    model= CNN(in_channels=3, num_class=10,num_filters=nf,kernel_sizes=ks,fc_neurons=fc,batch_norm=bn,dropout=dp,filter_multiplier=fm,activation=act).to(device)
    print("model ini==============================================================")
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer=opti(model,opt,lr)
    print("done")
    # Train Network
    for epoch in range(epochs):
        print('epoch enter')
        # Set the model to training mode
        model.train()

        for batch_idx, (data, targets) in enumerate(train_loader):
            # Get data to cuda if possible
            data = data.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            # forward
            scores = model(data)
            loss = criterion(scores, targets)

            # backward

            loss.backward()

            # gradient descent or adam step
            optimizer.step()  
            del data
            del targets
            
        # Calculate the test accuracy
        train_loss,train_acc = calculate_accuracy(model, train_loader,criterion)
        val_loss,val_acc = calculate_accuracy(model, val_loader,criterion)
        test_loss,test_acc = calculate_accuracy(model, test_loader,criterion)

        torch.cuda.empty_cache()
        # Log the metrics to WandB
        wandb.log({'epoch': epoch+1,'loss':loss.item(), 'train_loss': loss.item(),'test_loss':test_loss,'val_loss':val_loss,'test_acc': test_acc,'train_acc': train_acc,'val_acc': val_acc})


    # Save the best model
    wandb.save('model.h5')
    return


In [ ]:

# Initialize the WandB sweep
sweep_id = wandb.sweep(sweep_config, project='CS6910_Assignment_2_Q2')
wandb.agent(sweep_id, function=train,count=5)


Create sweep with ID: 9y3gr37b
Sweep URL: https://wandb.ai/cs22s015/CS6910_Assignment_2_Q2/sweeps/9y3gr37b


wandb: Agent Starting Run: ek21vk8h with config:
wandb: 	activation: LeakyRelu
wandb: 	batch_norm: True
wandb: 	batch_size: 128
wandb: 	data_aug: True
wandb: 	dropout: 0
wandb: 	fc_neurons: 64
wandb: 	filter_multiplier: 1
wandb: 	kernel_sizes: [3, 3, 3, 3, 3]
wandb: 	learning_rate: 0.0001
wandb: 	num_filters: 16
wandb: 	optimizer: nadam


7999
2000
data loaded ====================================================
ok pool5
r 300
r 151
r 76
r 39
r 20
ok flatten
11
model ini==============================================================
in opti
exit opti
done
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▄▃▅▅▅▃▂▁▇▄▁▄▂▂
test_acc,▁▃▄▄▅▆▆▇▇▇█████
test_loss,█▆▅▄▃▃▂▂▂▂▂▁▁▁▁
train_acc,▁▂▄▄▅▆▆▇▆▇▇▇▇██
train_loss,█▄▃▅▅▅▃▂▁▇▄▁▄▂▂
val_acc,▁▃▄▅▅▆▆▆▆▆▆▇███
val_loss,█▆▅▄▄▃▂▂▃▂▂▂▁▁▁
epoch,15
loss,1.71352
test_acc,36.55


wandb: Agent Starting Run: rw0b5n9l with config:
wandb: 	activation: Mish
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	data_aug: True
wandb: 	dropout: 0
wandb: 	fc_neurons: 32
wandb: 	filter_multiplier: 1
wandb: 	kernel_sizes: [7, 5, 5, 3, 3]
wandb: 	learning_rate: 0.001
wandb: 	num_filters: 8
wandb: 	optimizer: adam


...

wandb: Agent Starting Run: 77cwcf95 with config:
wandb: 	activation: Gelu
wandb: 	batch_norm: True
wandb: 	batch_size: 128
wandb: 	data_aug: False
wandb: 	dropout: 0
wandb: 	fc_neurons: 32
wandb: 	filter_multiplier: 0.5
wandb: 	kernel_sizes: [7, 5, 5, 3, 3]
wandb: 	learning_rate: 0.0005
wandb: 	num_filters: 4
wandb: 	optimizer: sgd


7999
2000
data loaded ====================================================


Run 77cwcf95 errored: TypeError('empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:\n * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)\n * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)\n')
wandb: ERROR Run 77cwcf95 errored: TypeError('empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:\n * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)\n * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device

7999
2000
data loaded ====================================================
ok pool5
r 300
r 149
r 74
r 37
r 19
ok flatten
10
model ini==============================================================
in opti
exit opti
done
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,▇█▅▅▆▇▅▄▃▁▃▄▃▁▁
test_acc,▁▄▅▇▇▆▇▇▇▇▇▇███
test_loss,█▆▄▄▃▃▂▂▂▂▂▂▁▂▂
train_acc,▁▂▃▄▄▄▅▅▅▅▆▆▇▇█
train_loss,▇█▅▅▆▇▅▄▃▁▃▄▃▁▁
val_acc,▁▃▅▆▅▆▇▇▇▇▇████
val_loss,█▆▄▄▃▃▂▂▂▂▂▁▁▂▁
epoch,15
loss,1.63135
test_acc,35.6


In [ ]:

# Initialize the WandB sweep
sweep_id = wandb.sweep(sweep_config, project='CS6910_Assignment_2_Q2')
wandb.agent(sweep_id, function=train,count=5)


Create sweep with ID: o095cvvz
Sweep URL: https://wandb.ai/cs22s015/CS6910_Assignment_2_Q2/sweeps/o095cvvz


wandb: Agent Starting Run: xksdas72 with config:
wandb: 	activation: Mish
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	data_aug: False
wandb: 	dropout: 0
wandb: 	fc_neurons: 128
wandb: 	filter_multiplier: 1
wandb: 	kernel_sizes: [5, 5, 5, 5, 5]
wandb: 	learning_rate: 0.001
wandb: 	num_filters: 16
wandb: 	optimizer: adam


7999
2000
data loaded ====================================================
ok pool5
r 300
r 150
r 75
r 37
r 18
ok flatten
9
model ini==============================================================
in opti
exit opti
done
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter
epoch enter


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▆▅▅▄▄▄▆▃▂▅▃▂▂▁
test_acc,▁▄▃▃▆▇▇▇███▇▇▆▆
test_loss,█▅▇▅▂▂▁▁▁▁▂▆▄▆▆
train_acc,▁▂▂▂▃▄▄▄▅▆▆▆▇▇█
train_loss,█▆▅▅▄▄▄▆▃▂▅▃▂▂▁
val_acc,▁▃▄▃▆▆▇▇▇██▇▇▇▆
val_loss,▇▅█▄▂▂▁▁▂▂▂▅▆▇█
epoch,15
loss,1.12455
test_acc,33.2


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 2kmc4vff with config:
wandb: 	activation: Gelu
wandb: 	batch_norm: True
wandb: 	batch_size: 128
wandb: 	data_aug: False
wandb: 	dropout: 0.2
wandb: 	fc_neurons: 128
wandb: 	filter_multiplier: 0.5
wandb: 	kernel_sizes: [7, 5, 5, 3, 3]
wandb: 	learning_rate: 0.0005
wandb: 	num_filters: 16
wandb: 	optimizer: adam


Thread WriterThread:
Traceback (most recent call last):
  File "/home/vplab/Anushka/Dl/lib/python3.8/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/home/vplab/Anushka/Dl/lib/python3.8/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/home/vplab/Anushka/Dl/lib/python3.8/site-packages/wandb/sdk/internal/internal.py", line 380, in _process
    self._wm.write(record)
  File "/home/vplab/Anushka/Dl/lib/python3.8/site-packages/wandb/sdk/internal/writer.py", line 154, in write
    write_handler(record)
  File "/home/vplab/Anushka/Dl/lib/python3.8/site-packages/wandb/sdk/internal/writer.py", line 131, in _write
    self.open()
  File "/home/vplab/Anushka/Dl/lib/python3.8/site-packages/wandb/sdk/internal/writer.py", line 71, in open
    self._ds.open_for_write(self._settings.sync_file)
  File "/home/vplab/Anushka/Dl/lib/python3.8/site-packages/wandb/sdk/internal/datastore.py", line 87, in open_f

In [ ]:

# Initialize the WandB sweep
sweep_id = wandb.sweep(sweep_config, project='CS6910_Assignment_2_Q2')
wandb.agent(sweep_id, function=train,count=5)


In [ ]:

# Initialize the WandB sweep
sweep_id = wandb.sweep(sweep_config, project='CS6910_Assignment_2_Q2')
wandb.agent(sweep_id, function=train,count=5)
